## Steps 
- Import cost & energy library 
- calculate energy and Li content for standard cells (NMC622, NMC811, LFP, SSB) 
- take assumption for 2030 distribution
- calculate extrem in case only one chemistry exists 

In [2]:
import math
import sys
sys.path.insert(0, '/home/matthias/Documents/GitHub/Cell_model_energy_costs')
import cell_model as Cellmodel
import numpy as np

## Define materials 

In [7]:
# Active materials
NMC811_cathode = Cellmodel.Activematerial_cathode("NMC811" , 195, 3.86)
NMC622_cathode = Cellmodel.Activematerial_cathode("NMC622" , 181, 3.86)
LFP_cathode = Cellmodel.Activematerial_cathode("LFP", 160, 3.3)
Gr = Cellmodel.Activematerial_anode("Gr", 344 , 0.17)
LiM = Cellmodel.Activematerial_anode("LiMetall", 3862, 0.0)

## Current Collectors, Eleytrolytes and Separators

In [11]:
### Others
## Currentcollectors 
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)


## Electrolyte 
LP40_standard = Cellmodel.Electrolyte_liquid("LiPF6" , 
                                             "EC:DMC", 
                                             1.1, # molar concentration mol/L 
                                             151.9 # molarmass salt 
                                            )

LLZO_elyte = Cellmodel.Electrolyte_solid("LLZO",
                                         838.5, # molarmass electrolyte
                                         6.94, # molarmass
                                         7, # index Li in LLZO
                                         0.20, # fraction of SE in pos electrode
                                         5.1 # density in g cm-3
                                        )

Elyte_liquid = Cellmodel.Electrolyte_liquid("","",0,0) # need to define as a empty container for the function ...
                                                        # to work 
# Separator
Separator_standard = Cellmodel.Separator("PP+Al" , 
                                         12e-4 , 
                                         0.44 , 
                                         1.18
                                        )
Separator_LLZO = Cellmodel.Separator("LLZO", 
                                     40e-4, # thickness
                                     0.7, # porosity
                                     5.1 # density g cm-3
                                    ) 

## Electrodes

In [14]:
# Electrods 
positive_NMC811 = Cellmodel.Electrodecomposition_cathode_opt1(NMC811_cathode.name , 
                                                              3.3 , # areal capacity 
                                                              0.95 , # active fraction 
                                                              3.4, # density electrode
                                                              NMC811_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5
positive_NMC811_ASSB = Cellmodel.Electrodecomposition_cathode_opt1(NMC811_cathode.name , 
                                                              3.3 , 
                                                              (positive_NMC811.active_frac - 
                                                               LLZO_elyte.pos_electrode_fraction), 
                                                              3.4, 
                                                              NMC811_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

positive_NMC622 = Cellmodel.Electrodecomposition_cathode_opt1(NMC622_cathode.name , 
                                                              3.3 , 
                                                              0.95 , 
                                                              3.4, 
                                                              NMC622_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

positive_NMC622_ASSB = Cellmodel.Electrodecomposition_cathode_opt1(NMC622_cathode.name , 
                                                              3.3 , 
                                                              (positive_NMC622.active_frac - 
                                                               LLZO_elyte.pos_electrode_fraction), 
                                                              3.4, 
                                                              NMC622_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

positive_LFP = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name , 
                                                           2.5 , 
                                                           0.95 ,
                                                           2.5, 
                                                           LFP_cathode
                                                          ) # NMC = 3.4 / LFP = 2.5

positive_LFP_ASSB = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name , 
                                                              2.5 , 
                                                              (positive_LFP.active_frac - 
                                                               LLZO_elyte.pos_electrode_fraction), 
                                                              3.4, 
                                                              LFP_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

negative_Gr = Cellmodel.Electrodecomposition_anode_opt1(Gr.name , 
                                                        3.3*1.1 , 
                                                        0.965 , 
                                                        1.6, 
                                                        Gr
                                                       )

negative_LiM = Cellmodel.Electrodecomposition_anode_opt1(LiM.name , 
                                                         10.32 , 
                                                         1.00 , 
                                                         0.543, 
                                                         LiM
                                                        ) # 50µm == 10.32 mAh/cm2

## Combining all into cells

In [16]:
### Total cells
# Graphite Anode
NMC811Gr_cell = Cellmodel.Pouch("NMC811-Gr_P", 
                                positive_NMC811, 
                                negative_Gr, 
                                Separator_standard , 
                                LP40_standard, 
                                1.7, #  electrolyte capacity ratio  ml/Ah 
                                30, 
                                40, 
                                35 , # layers
                                113e-4, # pouch thickness
                                1.8 , # pouch density 
                                1.0, # pouchclearance
                                2 , # terminal height 
                                5 , # terminal width 
                                15e-4, # therminal thickness
                                2.76 , # terminal density pos  
                                8.96, # terminal density neg 
                                10, # extra mass 
                                0.93, # 
                                NMC811_cathode,
                                Gr
                                )

NMC622Gr_cell = Cellmodel.Pouch("NMC622-Gr_P", 
                                    positive_NMC622, 
                                    negative_Gr, 
                                    Separator_standard , 
                                    LP40_standard, 
                                    1.7, #  electrolyte capacity ratio  ml/Ah 
                                    30, 
                                    10, 
                                    35 , # layers
                                    113e-4, # pouch thickness
                                    1.8 , # pouch density 
                                    1.0, # pouchclearance
                                    2 , # terminal height 
                                    5 , # terminal width 
                                    15e-4, # therminal thickness
                                    2.76 , # terminal density pos  
                                    8.96, # terminal density neg 
                                    10, # extra mass 
                                    0.93, # 
                                    NMC622_cathode,
                                    Gr
                                     )

LFPGr_cell = Cellmodel.Pouch("LFP-Gr_P", 
                                    positive_LFP, 
                                    negative_Gr, 
                                    Separator_standard , 
                                    LP40_standard, 
                                    1.7, #  electrolyte capacity ratio  ml/Ah 
                                    30, 
                                    10, 
                                    35 , # layers
                                    113e-4, # pouch thickness
                                    1.8 , # pouch density 
                                    1.0, # pouchclearance
                                    2 , # terminal height 
                                    5 , # terminal width 
                                    15e-4, # therminal thickness
                                    2.76 , # terminal density pos  
                                    8.96, # terminal density neg 
                                    10, # extra mass 
                                    0.93, # 
                                    LFP_cathode,
                                    Gr
                                  )

# Li metall Anode
NMC811LiM_cell = Cellmodel.Pouch("NMC811-LiM_P", 
                                    positive_NMC811_ASSB, 
                                    negative_LiM, 
                                    Separator_LLZO , 
                                    LLZO_elyte, 
                                    0, #  electrolyte capacity ratio  ml/Ah 
                                    30, 
                                    10, 
                                    35 , # layers
                                    113e-4, # pouch thickness
                                    1.8 , # pouch density 
                                    1.0, # pouchclearance
                                    2 , # terminal height 
                                    5 , # terminal width 
                                    15e-4, # therminal thickness
                                    2.76 , # terminal density pos  
                                    8.96, # terminal density neg 
                                    10, # extra mass 
                                    0.93, # 
                                    NMC811_cathode,
                                    LiM
                                      )

NMC622LiM_cell = Cellmodel.Pouch("NMC622-LiM_P", 
                                        positive_NMC622_ASSB, 
                                        negative_LiM, 
                                        Separator_LLZO , 
                                        LLZO_elyte,
                                        0, #  electrolyte capacity ratio  ml/Ah 
                                        30, 
                                        10, 
                                        35 , # layers
                                        113e-4, # pouch thickness
                                        1.8 , # pouch density 
                                        1.0, # pouchclearance
                                        2 , # terminal height 
                                        5 , # terminal width 
                                        15e-4, # therminal thickness
                                        2.76 , # terminal density pos  
                                        8.96, # terminal density neg 
                                        10, # extra mass 
                                        0.93, # 
                                        NMC622_cathode,
                                        LiM
                                      )

LFPLiM_cell = Cellmodel.Pouch("LFP-LiM_P", 
                                        positive_LFP_ASSB, 
                                        negative_LiM, 
                                        Separator_LLZO , 
                                        LLZO_elyte, 
                                        0, #  electrolyte capacity ratio  ml/Ah 
                                        30, 
                                        10, 
                                        35 , # layers
                                        113e-4, # pouch thickness
                                        1.8 , # pouch density 
                                        1.0, # pouchclearance
                                        2 , # terminal height 
                                        5 , # terminal width 
                                        15e-4, # therminal thickness
                                        2.76 , # terminal density pos  
                                        8.96, # terminal density neg 
                                        10, # extra mass 
                                        0.93, # 
                                        LFP_cathode,
                                        LiM
                                   )

In [18]:
NMC811LiM_cell.energy

995.0925599999999

In [19]:
NMC811LiM_cell.capacity

257.796

In [20]:
NMC811Gr_cell.energy

951.2672399999999

In [21]:
NMC811Gr_cell.capacity

257.796

In [22]:
negative_LiM.thickness_el_a

0.004921161279616378



## Get the Masses

In [7]:
positives = [positive_NMC811, positive_NMC622, positive_LFP]
negatives = [negative_Gr, negative_LiM]

cathodes = [NMC811_cathode, NMC622_cathode, LFP_cathode]

cells = [NMC811Gr_cell, NMC622Gr_cell, LFPGr_cell]

In [8]:
## Masses
cellmass_NMC811Gr, Housing_mass_NMC811Gr = Cellmodel.getMass_cylindrical_total(positive_NMC811, negative_Gr, Al, Cu, Separator_standard, NMC811Gr_cell, LP40_standard)
cellmass_NMC622Gr, Housing_mass_NMC622Gr = Cellmodel.getMass_cylindrical_total(positive_NMC622, negative_Gr, Al, Cu, Separator_standard, NMC622Gr_cell, LP40_standard)
cellmass_LFPGr, Housing_mass_LFPGr = Cellmodel.getMass_cylindrical_total(positive_LFP, negative_Gr, Al, Cu, Separator_standard, LFPGr_cell, LP40_standard)

## Cathode Masses
# Cylindrical
cathode_mass_NMC811 = Cellmodel.getMass_cathode(positive_NMC811) * NMC811Gr_cell.jr_area * 2
cathode_mass_NMC622 = Cellmodel.getMass_cathode(positive_NMC622) * NMC622Gr_cell.jr_area * 2
cathode_mass_LFP = Cellmodel.getMass_cathode(positive_LFP) * LFPGr_cell.jr_area * 2

## Seperator and Elyte
Separator_mass = Cellmodel.getMass_separator(Separator_standard)

pos_elec_materials = np.zeros((3,5))
neg_elec_materials = np.zeros((3,2))



In [9]:
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)

### Materials
## Cylindrical
i = 0
for positive in positives:
    Elyte_mass = Cellmodel.getMass_electrolyte(LP40_standard, cells[i])
    Ni, Co, Mn, Li, Al = Cellmodel.getMass_elements_cyl_c(positives[i], Al, cathodes[i], cells[i])
    Gr, Cu, Binder_a = Cellmodel.getMass_elements_cyl_a(negative_Gr, Cu, cells[i])
    pos_elec_materials[i,] = [Ni, Co, Mn, Li, Al]
    neg_elec_materials[i,] = [Gr, Cu]
    i = i+1
    Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
    Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)
    

In [10]:
pos_elec_materials

array([[10.64429675,  1.33053709,  1.33053709,  1.55229328,  0.        ],
       [ 8.35118991,  2.78372997,  2.78372997,  1.62384248,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.85997496,  0.        ]])

In [11]:
Elyte_mass

(7.152032293594514, 0.9695140910370436)

In [13]:
cells[0].name_cylindrical

'NMC811-Gr_Cyl'

g / mol * mol / L * L = g

Molare Masse * concentration * Elektrolytvolume = Mass Salz

Masse Electrolyte [g] * Massenanteil Lisalz [%g]